## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-27-22-41-42 +0000,nyt,"How Oil, Drugs and Immigration Fueled Trump’s ...",https://www.nytimes.com/2025/12/27/us/politics...
1,2025-12-27-22-40-04 +0000,nypost,Cali abandons lawsuit challenging Trump decisi...,https://nypost.com/2025/12/27/us-news/cali-aba...
2,2025-12-27-22-38-09 +0000,nypost,"Couple who tried to sell baby for $1,000 and a...",https://nypost.com/2025/12/27/us-news/couple-s...
3,2025-12-27-22-38-04 +0000,latimes,A new view of two critical days that set the s...,https://www.latimes.com/california/story/2025-...
4,2025-12-27-22-35-46 +0000,bbc,UK restricts DR Congo visas over migrant retur...,https://www.bbc.com/news/articles/c5y21xlxng2o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2391/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
4,trump,19
27,new,13
107,year,11
121,one,9
50,christmas,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
123,2025-12-27-03-30-11 +0000,nypost,Trump-Kennedy Center filing $1M lawsuit agains...,https://nypost.com/2025/12/26/us-news/trump-ke...,56
113,2025-12-27-07-34-22 +0000,nyt,Zelensky Will Meet With Trump Over the Weekend...,https://www.nytimes.com/2025/12/26/world/europ...,54
105,2025-12-27-09-04-51 +0000,nypost,"Russian drones, missiles pound Ukraine before ...",https://nypost.com/2025/12/27/world-news/russi...,54
108,2025-12-27-08-44-24 +0000,wapo,"Zelensky says he will meet Trump soon, citing ...",https://www.washingtonpost.com/world/2025/12/2...,54
128,2025-12-27-03-00-00 +0000,wsj,The U.S. is fast-tracking power projects for d...,https://www.wsj.com/politics/policy/ai-state-f...,54


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
123,56,2025-12-27-03-30-11 +0000,nypost,Trump-Kennedy Center filing $1M lawsuit agains...,https://nypost.com/2025/12/26/us-news/trump-ke...
88,48,2025-12-27-10-32-03 +0000,cbc,"Thailand, Cambodia sign new ceasefire deal to ...",https://www.cbc.ca/news/world/thailand-cambodi...
105,35,2025-12-27-09-04-51 +0000,nypost,"Russian drones, missiles pound Ukraine before ...",https://nypost.com/2025/12/27/world-news/russi...
128,35,2025-12-27-03-00-00 +0000,wsj,The U.S. is fast-tracking power projects for d...,https://www.wsj.com/politics/policy/ai-state-f...
24,32,2025-12-27-19-46-16 +0000,cbc,"Jeffrey R. Holland, next in line to lead Churc...",https://www.cbc.ca/news/world/jeffrey-r-hollan...
135,29,2025-12-27-00-57-52 +0000,bbc,"One general, one mathematician: The men compet...",https://www.bbc.com/news/articles/cpwk9grv290o...
55,26,2025-12-27-15-00-00 +0000,nypost,Test your 2025 news knowledge with The Post’s ...,https://nypost.com/2025/12/27/lifestyle/test-y...
108,24,2025-12-27-08-44-24 +0000,wapo,"Zelensky says he will meet Trump soon, citing ...",https://www.washingtonpost.com/world/2025/12/2...
130,22,2025-12-27-02-00-00 +0000,wsj,"A crop of moderate Democrats, including Sens. ...",https://www.wsj.com/politics/elections/democra...
14,22,2025-12-27-21-10-38 +0000,nypost,Ukrainian anti-corruption police raid Kyiv par...,https://nypost.com/2025/12/27/world-news/ukrai...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
